In [11]:
!pip install mediapipe opencv-python

  Using cached mediapipe-0.10.14-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached jax-0.4.35-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.4.35-cp312-cp312-win_amd64.whl.metadata (1.0 kB)
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.5.1-py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached ml_dtypes-0.5.0-cp312-cp312-win_amd64.whl.metadata (22 kB)
Using cached mediapipe-0.10.14-cp312-cp312-win_amd64.whl (50.8 MB)
Using cached sounddevice-0.5.1-py3-none-win_amd64.whl (363 kB)
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
Using cached jax-0.4.35-py3-none-any.whl (2.2 MB)
Using cached jaxlib-0.4.35-cp312-cp312-win_amd64.whl (56.6 MB)
Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl (45.5 MB)
Using cached ml_dtypes-0.5.0-cp312-cp312-win_amd64.whl (213 kB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Zachary\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [12]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

ModuleNotFoundError: No module named 'mediapipe'

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
# Function to calculate angles between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point
    c = np.array(c)  # Last point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

In [ ]:
# Push-Up Counter Logic
def pushup_counter_logic(landmarks, counter, stage):
    # Get relevant joints
    shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
             landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
             landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    
        # Calculate angle at elbow
    angle = calculate_angle(shoulder, elbow, wrist)
    
    # Stage determination (going down vs coming up)
    if angle > 160:
        stage = "down"
    if angle < 30 and stage == 'down':
        stage = "up"
        counter += 1
    
    return counter, stage

# Bicep Curl Counter Logic
def bicep_curl_counter_logic(landmarks, counter, stage):
    
    shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
             landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
             landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    
    angle = calculate_angle(shoulder, elbow, wrist)
    
    if angle > 160:
        stage = "down"
    if angle < 30 and stage == 'down':
        stage = "up"
        counter += 1
        
    return counter, stage

# Sit-Up Counter Logic
def situp_counter_logic(landmarks, counter, stage):
    
    hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
           landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
             landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    
    angle = calculate_angle(hip, knee, ankle)
    
    if angle > 170:
        stage = "down"
    if angle < 90 and stage == 'down':
        stage = "up"
        counter += 1
        
    return counter, stage


In [ ]:
def main():
    # Variables for counting and stage
    counter = 0
    stage = None
    exercise = 'push-up'  
    
    
    cap = cv2.VideoCapture(0)
    
    
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            
           
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            
            
            results = pose.process(image)
            
            
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            
            try:
                landmarks = results.pose_landmarks.landmark
                
                
                if exercise == 'push-up':
                    counter, stage = pushup_counter_logic(landmarks, counter, stage)
                elif exercise == 'bicep-curl':
                    counter, stage = bicep_curl_counter_logic(landmarks, counter, stage)
                elif exercise == 'sit-up':
                    counter, stage = situp_counter_logic(landmarks, counter, stage)
                
                
                
                cv2.putText(image, f'Counter: {counter}', (10, 50), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, f'Stage: {stage}', (10, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
            except:
                pass
            
            
            draw_landmarks(image, results)
            
            
            cv2.imshow('Exercise Feed', image)
            
            
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()


main()